![EvalML Logo](images/evalml_logo.png)


# What is EvalML?
EvalML is an AutoML library that builds, optimizes, and evalutes machine learning pipelines using domain-specific objective functions.


Combined with [Featuretools](https://featuretools.featurelabs.com) and [Compose](https://compose.featurelabs.com), EvalML can be used to create end-to-end machine learning solutions for classification and regression problems. 


# Quick Start

In [1]:
import evalml

## Load Data

First, we load in the features and outcomes we want to use to train our model

In [2]:
X, y = evalml.demos.load_breast_cancer()

## Configure search

EvalML has many options to configure the pipeline search. At the minimum, we need to define an objective function. For simplicity, we will use the F1 score in this example. However, the real power of EvalML is in using domain-specific [objective functions](objectives/overview.html) or [building your own](objectives/custom_objectives.html).

In [3]:
clf = evalml.AutoClassifier(objective="f1",
                            max_pipelines=5)

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

In [4]:
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, test_size=.2)

When we call `.fit()`, the search for the best pipeline will begin. 

In [5]:
clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for F1. Greater score is better.

Searching up to 5 pipelines. No time limit is set. Set one using max_time parameter.

Possible model types: linear_model, xgboost, random_forest

Testing Random Forest w/ imputation: 100%|██████████| 5/5 [00:24<00:00,  5.00s/it]               

✔ Optimization finished


## See Pipeline Rankings

After the search is finished we can view all of the pipelines searched, ranked by score. Internally, EvalML performs [cross validation](automl/guardrails.html) to score the pipelines. If it notices, a high variance across cross validation folds, it will warn you.

In [6]:
clf.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,2,LogisticRegressionPipeline,0.975637,False,"{'penalty': 'l2', 'C': 8.444214828324364, 'imp..."
1,0,RFClassificationPipeline,0.972001,False,"{'n_estimators': 569, 'max_depth': 22, 'impute..."
2,4,RFClassificationPipeline,0.963048,False,"{'n_estimators': 609, 'max_depth': 7, 'impute_..."
3,3,RFClassificationPipeline,0.961230,False,"{'n_estimators': 369, 'max_depth': 10, 'impute..."
4,1,XGBoostPipeline,0.959500,False,"{'eta': 0.5928446182250184, 'min_child_weight'..."


## Describe pipeline

If we are interested in see more details about the pipeline we can describe it using the `id` from the rankings table

In [9]:
clf.describe_pipeline(3)

************************
* Pipeline Description *
************************

Pipeline Name: Random Forest w/ imputation
Model type: random_forest
Objective: F1 (greater is better)
Total training time (including CV): 6.0 seconds

Parameters
• n_estimators: 369
• max_depth: 10
• impute_strategy: median
• percent_features: 0.39053789021977287

Cross Validation
               F1  Precision  Recall   AUC  Log Loss  # Training  # Testing
0           0.953      0.948   0.953 0.990     0.122     303.000    152.000
1           0.957      0.978   0.957 0.991     0.122     303.000    152.000
2           0.974      0.969   0.974 0.992     0.104     304.000    151.000
mean        0.961      0.965   0.961 0.991     0.116     303.333    151.667
std         0.011      0.015   0.011 0.001     0.010       0.577      0.577
coef of var 0.012      0.016   0.012 0.001     0.089       0.002      0.004


## Select Best pipeline
We can now select best pipeline and score it on our holdout data

In [8]:
pipeline = clf.best_pipeline
pipeline.score(X_holdout, y_holdout)

0.9589041095890412

# Getting Started
[What is EvalML](self)

[Install](install)

# Objective functions

[Overview](objectives/overview)

[Fraud Prediction](demos/fraud)

[Lead Scoring](demos/lead_scoring)

[Defining Custom Objectives](objectives/custom_objectives)

# Automated Machine Learning

[Setting up pipeline search](automl/pipeline_search)

[Exploring search results](automl/search_results)

[Avoiding Overfitting](automl/guardrails)

[Regression Example](automl/regression_example)

# Resources
[Roadmap](roadmap)

[API Reference](api_reference)